# Init

In [71]:
import anthropic
from vertexai.generative_models import GenerativeModel, SafetySetting
from pprint import pprint as pp
import json
import time
from dotenv import load_dotenv
import re
import math
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import PyPDF2

In [72]:
model = "claude-3-5-sonnet-20241022"

In [73]:
load_dotenv()
client = anthropic.Anthropic()

# Prompts

## Splitted text prompt

### Format

In [74]:
split_text_format = '''{
    question: str,
    textPart: str,
    answers: [
        {
            answer: str,
            isCorrect: boolean,
        }
    ]
}'''

In [75]:
topics = '''1. questions about the personalities of the characters
2. questions about the appearance or look of characters
3. what the characters did in certain situations
4. what the characters did in certain situations, what they thought or felt
5. the attitude of some characters to other characters
6. names of cities or locations where the events took place
7. what are the key characters'''

### Prompt

In [76]:
split_text_prompt = '''Your task is to read a part of the book carefully and create only ONE question for the children who have read this part. 
Question should have 4 answer choices with 1 correct answer.

Here are some topics to create a question:
1. questions about the personalities of the characters
2. questions about the appearance or look of characters
3. what the characters did in certain situations
4. what the characters did in certain situations, what they thought or felt
5. the attitude of some characters to other characters
6. names of cities or locations where the events took place
7. what are the key characters

Try to make question and answer choices more similar to the text itself.
Avoid creating tricky questions.

The question must be in this json format:
```
{}
```

The textPart field of the question should contain a part of the text from which you took the question

Here is a part of a book:
{}

The question must be in the same language as the book
'''

In [77]:
gen_prompt_template = '''You are an experienced educational content creator specializing in reading comprehension exercises for children. Your task is to create a single multiple-choice question based on a given book excerpt.

Here's the book excerpt you'll be working with:

<book_excerpt>
{}
</book_excerpt>

Now, let's review the list of topics that may be relevant to the question you'll create:

<topics_list>
{}
</topics_list>

Your task is to carefully read this excerpt and create ONE question for children who have read this part. Follow these guidelines:

1. Create a question that is closely related to the content of the text.
2. Generate 4 answer choices, with only 1 correct answer.
3. Ensure that the question and answer choices are similar in language and style to the text itself.
4. Avoid creating tricky or intentionally misleading questions or answers.
5. The question must be in the same language as the book excerpt.

You must format your output as a JSON object. Here's the required format:

<json_format>
{}
</json_format>

Important: The "textPart" field of the question should contain the specific part of the text from which you derived the question.

Before providing your final answer, wrap your thought process inside question_development tags. Consider the following steps:

1. Write down 2-3 key quotes from the excerpt that could be potential question sources.
2. List out 3-4 potential topics from the provided topic list that relate to these quotes.
3. For each potential question:
   - Formulate a clear and concise question.
   - Write down the correct answer and three plausible but incorrect answers.
   - Evaluate the question based on clarity, relevance, and difficulty level.
4. Choose the best question based on your evaluation.
5. Select the relevant text part that corresponds to your chosen question.

After your thought process, provide the final JSON output with your question, answers, and relevant text part.'''

In [78]:
gen_prompt_template_2 = '''You are an experienced educational content creator specializing in reading comprehension exercises for children. Your task is to create a single, clear, and specific multiple-choice question based on a given book excerpt.

First, carefully read the following book excerpt:

<book_excerpt>
{}
</book_excerpt>

Now, review this list of topics that may be relevant to the question you'll create:

<topics_list>
{}
</topics_list>

Your goal is to create ONE question for children who have read this excerpt. Follow these guidelines:

1. The question must be closely related to the content of the text.
2. Generate 4 answer choices, with only 1 correct answer.
3. Use language and style similar to the text itself.
4. Avoid creating tricky or intentionally misleading questions or answers.
5. The question must be in the same language as the book excerpt.
6. Ensure the question is clear and specific, explicitly mentioning any relevant context from the excerpt.

Before creating your final question, wrap your reasoning process in <question_development> tags. Follow these steps:

1. Identify 2-3 key quotes from the excerpt that could be potential question sources. Write these quotes down verbatim.
2. List 3-4 relevant topics from the provided topic list that relate to these quotes.
3. For each potential question:
   - Formulate a clear and concise question, ensuring it includes specific context.
   - Write the correct answer and three plausible but incorrect answers.
   - Evaluate the question based on clarity, relevance, and difficulty level.
   - Consider the age-appropriateness of the question for children.
   - Assess how well the question aligns with the provided topics.
4. Choose the best question based on your evaluation.
5. Select the relevant text part that corresponds to your chosen question.
6. Explain why the chosen question is the best option, considering all factors evaluated.

After your question development process, provide the final output as a JSON object. Here's the required format:

<json_format>
{}
</json_format>

Remember, the "textPart" field should contain the specific part of the text from which you derived the question.'''

# Utils

## Get formatted splitted text prompt

In [79]:
def get_split_text_prompt(text_part: str):
    return split_text_prompt.format(split_text_format, text_part)

In [80]:
def get_gen_prompt(text_part: str):
    return gen_prompt_template_2.format(text_part, topics, split_text_format)

## Remove new lines

In [81]:
def remove_extra_newlines(text):
    return re.sub(r'\n+', '\n', text)

## Read PDF

In [82]:
def read_pdf(file_path: str):
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                page = reader.pages[page_num]
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

## Read EPUB

In [83]:
def read_epub(file_path):
    book = epub.read_epub(file_path)
    text_content = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_body_content(), 'html.parser')
            text_content.append(soup.get_text())

    text = '\n'.join(text_content)
    return remove_extra_newlines(text)

## Split text

In [84]:
def split_text_into_parts(text: str, num_parts: int):
    paragraphs = re.split('\n', text.strip())
    
    paragraphs_per_part = math.ceil(len(paragraphs) / (num_parts))
    
    parts = []
    current_part = ""
    
    for i, paragraph in enumerate(paragraphs):
        current_part += paragraph + "\n"
        if (i + 1) % paragraphs_per_part == 0 or (i + 1) == len(paragraphs):
            parts.append(current_part.strip())
            current_part = ""
    
    return parts

## Split text with overlap

In [85]:
def split_text_into_parts_with_overlap(text: str, num_parts: int):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text.strip())
    
    sentences_per_part = math.ceil(len(sentences) / num_parts)
    
    parts = []
    for i in range(num_parts):
        start_index = max(0, i * sentences_per_part - 1)
        end_index = min(len(sentences), (i + 1) * sentences_per_part + 1)
        
        part = " ".join(sentences[start_index:end_index])
        parts.append(part.strip())
    
    return parts

## Claude generate

In [86]:
def claude_generate_answer(model: str, prompt: str):
    response = client.messages.create(
        model=model,
        max_tokens=2000,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )

    return response

# Generate answers

In [94]:
text = read_epub('../books/vovchok-marko-vedmid-sestrychka-melasia1976.epub')
parts = split_text_into_parts(text, 10)
input_tokens = 0
output_tokens = 0
for part in parts:
    prompt = get_gen_prompt(part)
    response = claude_generate_answer(model, prompt)
    input_tokens += response.usage.input_tokens
    output_tokens += response.usage.output_tokens
    print(response.content[0].text)
    print("================================")

<question_development>
1. Key quotes from excerpt:
- "Ведмідь (Сестричка Мелася)"
- "Марко Вовчок"
- "Від Падолиста"

2. Relevant topics from list:
- Topic 7: what are the key characters
- Topic 6: names of cities or locations where the events took place

3. Potential questions analysis:
a) About the author:
- Question: "Хто є автором твору 'Ведмідь'?"
- This question is clear, specific, and based on directly stated information
- Age-appropriate for children
- Aligns with understanding basic text elements
- Easy to verify from the text

b) About the title:
- Question: "Яка назва твору?"
- Too simple and obvious
- Doesn't require comprehension

4. Best question selection:
I choose the author question because:
- It teaches children to pay attention to important text elements
- It's factual and unambiguous
- It helps children learn to identify and remember author names
- It's appropriate for reading comprehension exercises

5. Relevant text part:
"Ведмідь (Сестричка Мелася)
Марко Вовчок"


In [88]:
# print(response)

In [96]:
print(prompt)

You are an experienced educational content creator specializing in reading comprehension exercises for children. Your task is to create a single, clear, and specific multiple-choice question based on a given book excerpt.

First, carefully read the following book excerpt:

<book_excerpt>
Перше, то не диво здавалося Меласі, бо братік коханий і любий. як же його не боронити? А друге, то, мабуть, уважала Мелася за диво потішне й не нікчемне, бо як згадувала про його, зараз червоніла і зараз сміялася веселенько і, може, трохи хвастливенько.
– Меласю-сестричко! Коли ж я тебе боронитиму? – питав часом Михайло, як розпалений, оглядаючись, чи нема де чого страшного або трудного, щоб його знівечити. – Коли мені доведеться тебе визволяти?
І лягав спати Михайло, може, не раз з потайною надією, що от ведмідь знов одвідає, і вночі не спав, прислухавсь, і бравсь за рушницю, що собі покупив – тільки ще досі, чутно, не діждав патлатого він гостя-одвідача.
– О, братіку, братіку, не дай, Боже, щоб… Мати

In [90]:
# print(len(parts))

In [95]:
print(input_tokens)
print(output_tokens)

12288
5520


In [91]:
# text = read_epub('../books/kobylianska-olha-iulianivna-Valse-melancolique-melankholiynyy-vals2457.epub')
# parts = split_text_into_parts(text, 30)
# input_tokens = 0
# output_tokens = 0
# for part in parts:
#     prompt = get_gen_prompt(part)
#     response = claude_generate_answer(model, prompt)  
#     input_tokens += response.usage.input_tokens
#     output_tokens += response.usage.output_tokens
#     print(response.content[0].text)
#     print("================================")

<question_development>
1. Key quotes:
- "Не можу слухати меланхолійної музики."
- "Нас мешкало разом три товаришки."
- "Була вже майже укінченою артисткою і працювала саме над одним образом, який хотіла продати і поїхати до Італії"

2. Relevant topics:
- Topic 7: what are the key characters
- Topic 1: questions about the personalities of the characters
- Topic 3: what the characters did in certain situations

3. Potential questions evaluation:
a) About the roommates:
- Question: "Скільки товаришок мешкало разом?"
- Clear context from text
- Easy to verify from text
- Age-appropriate
- Straightforward comprehension question

b) About the painter:
- Question about her plans
- More complex, involves multiple details
- Might be too detailed for younger readers

4. Best question selection:
I choose the first question about the number of roommates because:
- It's clearly stated in the text
- It's a straightforward comprehension question
- It's age-appropriate
- It helps establish the basic s

In [93]:
print(input_tokens)
print(output_tokens)

50331
18222
